# Create register summary from process register file

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
ROOT_FOLDER = '/home/msdc/jcgarciaca/projects/thalon/OperationRegisters/TUV2002'

In [3]:
operation_time = pd.read_csv(os.path.join(ROOT_FOLDER, 'operation_time.csv'))
operation_time.head()

,ON,OFF,Duration
0,10/06/2020 10:35:25,10/06/2020 10:35:38,12.865913
1,10/06/2020 10:36:24,10/06/2020 10:36:32,8.643720
2,10/06/2020 10:36:47,10/06/2020 10:37:02,15.042624
3,10/06/2020 10:37:18,10/06/2020 10:37:23,5.190662
4,18/06/2020 14:50:19,18/06/2020 14:52:14,114.848579


In [4]:
process_register = pd.read_csv(os.path.join(ROOT_FOLDER, 'process_register_tmp.csv'))
process_register.head()

,ID,ON,OFF,Duration,Status
0,cll100 admin p3,01/02/2021 11:41:03,01/02/2021 11:54:22,799.349200,Completed
1,cll100 admin p3,01/02/2021 11:55:52,01/02/2021 11:57:02,70.425639,Completed
2,cll100 desarrollo p3,01/02/2021 12:32:03,01/02/2021 12:33:55,112.190667,Failed
3,cll100 desarrollo p3,01/02/2021 18:07:36,01/02/2021 18:20:35,779.387600,Failed
4,cll100 desarrollo p3,01/02/2021 18:22:52,01/02/2021 18:22:59,7.536354,Failed


In [5]:
operation_time.head()

,ON,OFF,Duration
0,10/06/2020 10:35:25,10/06/2020 10:35:38,12.865913
1,10/06/2020 10:36:24,10/06/2020 10:36:32,8.643720
2,10/06/2020 10:36:47,10/06/2020 10:37:02,15.042624
3,10/06/2020 10:37:18,10/06/2020 10:37:23,5.190662
4,18/06/2020 14:50:19,18/06/2020 14:52:14,114.848579


In [6]:
process_register.head()

,ID,ON,OFF,Duration,Status
0,cll100 admin p3,01/02/2021 11:41:03,01/02/2021 11:54:22,799.349200,Completed
1,cll100 admin p3,01/02/2021 11:55:52,01/02/2021 11:57:02,70.425639,Completed
2,cll100 desarrollo p3,01/02/2021 12:32:03,01/02/2021 12:33:55,112.190667,Failed
3,cll100 desarrollo p3,01/02/2021 18:07:36,01/02/2021 18:20:35,779.387600,Failed
4,cll100 desarrollo p3,01/02/2021 18:22:52,01/02/2021 18:22:59,7.536354,Failed


In [7]:
process_register['Fecha'] = process_register['ON'].apply(lambda x: x.split()[0])
process_register['Hora'] = process_register['ON'].apply(lambda x: x.split()[1])
process_register.head()

,ID,ON,OFF,Duration,Status,Fecha,Hora
0,cll100 admin p3,01/02/2021 11:41:03,01/02/2021 11:54:22,799.349200,Completed,01/02/2021,11:41:03
1,cll100 admin p3,01/02/2021 11:55:52,01/02/2021 11:57:02,70.425639,Completed,01/02/2021,11:55:52
2,cll100 desarrollo p3,01/02/2021 12:32:03,01/02/2021 12:33:55,112.190667,Failed,01/02/2021,12:32:03
3,cll100 desarrollo p3,01/02/2021 18:07:36,01/02/2021 18:20:35,779.387600,Failed,01/02/2021,18:07:36
4,cll100 desarrollo p3,01/02/2021 18:22:52,01/02/2021 18:22:59,7.536354,Failed,01/02/2021,18:22:52


In [8]:
def add_info(data_dict, zone, date, start, end):
    data_dict['Zona'].append(zone)
    data_dict['Fecha'].append(date)
    data_dict['Inicio'].append(start)
    data_dict['Fin'].append(end)
    return data_dict

def is_same(date_1, end_1, date_2, start_2):    
    datetime_1 = datetime.strptime(date_1 + ' ' + end_1, '%d/%m/%Y %H:%M:%S')
    datetime_2 = datetime.strptime(date_2 + ' ' + start_2, '%d/%m/%Y %H:%M:%S')
    delta = datetime_2 - datetime_1
    h = (delta.total_seconds()) // 3600
    return h < 3

In [9]:
data = {'Zona': [], 'Fecha': [], 'Inicio': [], 'Fin': []}
for i, g in process_register.groupby([(process_register['ID'] != process_register['ID'].shift()).cumsum()]):   
    if len(g) == 1:
        data = add_info(data, 
                 g.iloc[0]['ID'], 
                 g.iloc[0]['OFF'].split()[0], 
                 g.iloc[0]['ON'].split()[1], 
                 g.iloc[0]['OFF'].split()[1]
                )
    elif len(g) > 1:
        date_1_ON = g.iloc[0]['ON'].split()[0]
        date_1_OFF = g.iloc[0]['OFF'].split()[0]
        start_1 = g.iloc[0]['ON'].split()[1]
        end_1 = g.iloc[0]['OFF'].split()[1]
        
        date_2_ON = g.iloc[-1]['ON'].split()[0]
        date_2_OFF = g.iloc[-1]['OFF'].split()[0]
        start_2 = g.iloc[-1]['ON'].split()[1]
        end_2 = g.iloc[-1]['OFF'].split()[1]
        if is_same(date_1_OFF, end_1, date_2_ON, start_2):
            # merge
            data = add_info(data, 
                     g.iloc[0]['ID'], 
                     date_1_OFF, 
                     start_1, 
                     end_2
                    )
        else:
            # keep separeted
            data = add_info(data, 
                     g.iloc[0]['ID'], 
                     date_1_OFF, 
                     start_1, 
                     end_1
                    )
            data = add_info(data, 
                     g.iloc[-1]['ID'], 
                     date_2_OFF, 
                     start_2, 
                     end_2
                    )

In [10]:
df = pd.DataFrame.from_dict(data)
df.head()

,Zona,Fecha,Inicio,Fin
0,cll100 admin p3,01/02/2021,11:41:03,11:57:02
1,cll100 desarrollo p3,01/02/2021,12:32:03,12:33:55
2,cll100 desarrollo p3,01/02/2021,18:24:05,18:26:51
3,cll100 bagrario p3,01/02/2021,18:30:43,18:45:13
4,cll100 nestle p1,01/02/2021,19:00:52,19:09:38


In [11]:
df.to_csv(os.path.join(ROOT_FOLDER, 'summary.csv'), index=False)